In [1]:
from gurobipy import *

In [2]:
from asyncio.constants import SENDFILE_FALLBACK_READBUFFER_SIZE
import random
from random import seed
import numpy as np
from scipy.stats import multivariate_normal
class DirectSolve:
    def __init__(self, init:list, mean, cov, size, size_, beta):
        self.ship_num = len(init[2])
        self.stack_num = init[0]
        self.height = init[1]
        self.n_init = init[2]
        self.beta = beta
        self.size = size
        self.size_ = size_
        self.mean = mean
        self.cov = cov
        self.OC = []
        self.zk = []

        # 集合の定義
        self.O1 = [i+1 for i in range(self.ship_num)]
        self.S = [i+1 for i in range(self.stack_num)]
        self.H = [i+1 for i in range(self.height)]
        self.P = [i+1 for i in range(self.ship_num)]
        self.f = self.stack_num * self.height - sum(self.n_init)

        # 多変量正規分布からシナリオをsize個生成
        data_1 = np.random.multivariate_normal(mean, cov, size = self.size)

        O_ = np.argsort(data_1)
        O_ = (O_ + np.ones((size, self.ship_num)).astype(int)).tolist()
        self.O = O_
        # O=uncertain_set(self.ship_num, self.size)

    def get_penalty(self, result):
        self.result=result
        self.penalty=[]
        for k in self.O:
            OO=k
            a=0
            for j in range(self.stack_num):
                for i in range(1,self.height):
                    for i_ in range(i+1,self.height+1):
                        if self.result[i-1][j]!=0 and self.result[i_-1][j]!=0:
                            if OO.index(self.result[i-1][j])>OO.index(self.result[i_-1][j]):
                                # print(j+1,height-i+1,O)
                                a+=1
                                # print("penalty!")
                                    # print(i,i_,j+1,O)
                                break
            self.penalty.append(a)
        self.zk=[i for i, x in enumerate(self.penalty) if x-self.alpha_tmp>0]
        
        for zk_ in self.zk:
            if not zk_ in self.aaa:
                self.aaa.append(zk_)
                self.OC.append(O[zk_])
        return (self.zk,self.OC,self.penalty)

    def solve(self):
        self.model = Model("MIP1")
        a = []
        dic = {}
        for i in self.O:
            # print(i)
            if not i in a:
                a.append(i)
        for j in a:
            dic[tuple(j)] = self.O.count(j)
        # print("dic=", dic)
        x, c, d = {}, {}, {}
        for s in self.S:
            for h in self.H:
                for p in self.P:
                    x[s, h, p] = self.model.addVar(vtype = "B", name="x_" + str(s) + "_" + str(h) + "_" + str(p))

        for s in self.S:
            for h in range(2, len(self.H) + 1):
                for i, o in enumerate(self.O):
                    c[s, h, i] = self.model.addVar(vtype = "C", lb = 0, name = "c_" + str(s) + "_" + str(h) + "_" + str(i))

        for i in range(len(self.O)):
            d[i] = self.model.addVar(vtype = "C", lb = 0)
        alpha = self.model.addVar(vtype = "C", lb = 0, name = "alpha")
        u = self.model.addVar(vtype = "C", lb = 0, name = "u")

        self.model.update()
        self.model._vars = self.model.getVars()

        for p in self.P:
            self.model.addConstr(quicksum(x[s, h, p] for s in self.S for h in self.H) == self.n_init[p - 1])

        for s in self.S:
            for h in self.H:
                self.model.addConstr(quicksum(x[s, h, p] for p in self.P) <= 1)

        for s in self.S:
            for h in range(1, len(self.H)):
                self.model.addConstr(quicksum(x[s, h+1, p] for p in self.P) <= quicksum(x[s, h, p] for p in self.P))

        for s in self.S:
            for h in range(2, len(self.H)+1):
                for h_ in range(1, h):
                    for i, o in enumerate(dic):
                        for j, p in enumerate(o):
                            self.model.addConstr(c[s, h, i] >= dic[o]*(quicksum(x[s, h, k] for k in o[j:]) - quicksum(x[s, h_, k] for k in o[j:])))
        for i in range(len(dic)):
            self.model.addConstr(d[i] >= quicksum(c[s, h, i] for s in self.S for h in self.H if h != 1) - alpha)
        self.model.addConstr(u >= alpha + quicksum(d[i] for i in range(len(dic))) / ((1 - self.beta) * len(self.O)))

        self.model.setObjective(u)
        self.model._vars = self.model.getVars()
        # self.model.params.MIPFocus=2
        if self.f >= self.height:
            self.model.Params.TimeLimit = 7200
            self.model.optimize()
            if self.model.Status == GRB.OPTIMAL:
                x_opt, c_opt, alpha_opt, u_opt = self.get_optimal_sol()
                global result
                result = self.get_result(x_opt)
            u_opt = self.get_optimal_val()
            print("u=", u_opt)
        else:
            print("実行不可")

    def get_optimal_val(self):
        if self.model.Status==GRB.OPTIMAL:
            self.LB=self.model.ObjVal
            return self.model.ObjVal
        else:
            return None

    def get_optimal_sol(self):
        if self.model.Status == GRB.OPTIMAL:
            x_opt = {}
            c_opt = {}
            for var in self.model._vars:
                if "x_" in var.VarName:
                    # print(var)
                    s=int(var.VarName.split("_")[-3])
                    h=int(var.VarName.split("_")[-2])
                    p=int(var.VarName.split("_")[-1])
                    x_opt[s,h,p] = var.X
                if "c_" in var.VarName:
                    s=int(var.VarName.split("_")[-3])
                    h=int(var.VarName.split("_")[-2])
                    i=int(var.VarName.split("_")[-1])
                    c_opt[s,h,i] = var.X
                if "alpha" in var.VarName:
                    alpha_opt = var.X
                if "u" in var.VarName:
                    u_opt=var.X
            return (x_opt,c_opt,alpha_opt,u_opt)
        else:
            return None

    def get_result(self,x_tmp):
        self.x_tmp=x_tmp
        EPS=1.e-6
        self.result=np.zeros((self.height,self.stack_num))
        for (s,h,p) in self.x_tmp:
            if self.x_tmp[s,h,p]>EPS:
                self.result[self.height-h][s-1]=int(p)
        self.result=self.result.astype(int)
        return self.result
    
    # def get_penalty(self,result):
    #     if self.model.Status==GRB.OPTIMAL:
    #         self.result=result
    #         self.penalty=[]
    #         x,c,alpha,u=self.get_optimal_sol()
    #         for k in self.O:
    #             OO=k
    #             a=0
    #             for j in range(self.stack_num):
    #                 for i in range(1,self.height):
    #                     for i_ in range(i+1,self.height+1):
    #                         if self.result[i-1][j]!=0 and self.result[i_-1][j]!=0:
    #                             if OO.index(self.result[i-1][j])>OO.index(self.result[i_-1][j]):
    #                                 # print(j+1,height-i+1,O)
    #                                 a+=1
    #                                 # print("penalty!")
    #                                     # print(i,i_,j+1,O)
    #                                 break
    #             self.penalty.append(a)
    #         self.zk=[i for i, x in enumerate(self.penalty) if x-alpha>0]
    #         for zk_ in self.zk:
    #             self.OC.append(self.O[zk_])
    #         return (self.zk,self.OC,self.penalty)

class CVaR:
    def __init__(self, init:list, mean, cov, size, size_, beta):
        self.ship_num = len(init[2])
        self.stack_num = init[0]
        self.height = init[1]
        self.n_init = init[2]
        self.beta = beta
        self.size = size
        self.size_ = size_
        self.mean = mean
        self.cov = cov
        self.OC = []
        self.zk = []
        self.cutting_plane = []


        # 集合の定義
        self.O1 = set(i+1 for i in range(self.ship_num))
        self.S = set(i+1 for i in range(self.stack_num))
        self.H = set(i+1 for i in range(self.height))
        self.P = set(i+1 for i in range(self.ship_num))
        self.f = self.stack_num * self.height - sum(self.n_init)

        # 多変量正規分布からシナリオをsize個生成
        data_1 = np.random.multivariate_normal(mean, cov, size = self.size)

        # シナリオをソート
        O_ = np.argsort(data_1)
        O_ = (O_ + np.ones((size, self.ship_num)).astype(int)).tolist()
        self.O = O_

    # 不良配置数の計算
    def get_penalty(self, result):
        self.result = result
        self.penalty = []
        for k in self.O:
            p_ = 0
            for j in range(self.stack_num):
                for i in range(1,self.height):
                    for i_ in range(i+1, self.height+1):
                        if self.result[i-1][j]!=0 and self.result[i_-1][j] != 0:
                            if k.index(self.result[i-1][j]) > k.index(self.result[i_-1][j]):
                                # print(j+1,height-i+1,O)
                                p_ += 1
                                # print("penalty!")
                                    # print(i,i_,j+1,O)
                                break
            self.penalty.append(p_)
        self.zk = [i for i, x in enumerate(self.penalty) if x - self.alpha_tmp > 0]
        
        for zk_ in self.zk:
            self.OC = [self.O[zk_]]
        return (self.zk, self.OC, self.penalty)

    def solve(self):
        def add_cutting_plane(model, where):
            if where == GRB.callback.MIPSOL:
                c_var, x_var, x_tmp, d_var = {}, {}, {}, {}
                for var in self.model._vars:
                    if "u" in var.VarName:
                        u_var = var
                    if "alpha" in var.VarName:
                        alpha_var = var
                        self.alpha_tmp = model.cbGetSolution(var)
                    if "c_" in var.VarName:
                        s = int(var.VarName.split("_")[-3])
                        h = int(var.VarName.split("_")[-2])
                        i = int(var.VarName.split("_")[-1])
                        c_var[s, h, i] = var
                    if "d_" in var.VarName:
                        i = int(var.VarName.split("_")[-1])
                        d_var[i] = var
                    if "x_" in var.VarName:
                        s = int(var.VarName.split("_")[-3])
                        h = int(var.VarName.split("_")[-2])
                        p = int(var.VarName.split("_")[-1])
                        x_var[s, h, p] = var
                        x_tmp[s, h, p] = model.cbGetSolution(var)
                global result
                result = self.get_result(x_tmp)
                print(result)
                zk, OC, penalty = self.get_penalty(result)
                a = []
                dic={}
                for i in OC:
                    # print(i)
                    if not i in a:
                        a.append(i)
                for j in a:
                    dic[tuple(j)] = OC.count(j)
                print(len(zk), len(OC))
                for s in self.S:
                    for h in range(2, len(self.H) + 1):
                        for h_ in range(1, h):
                            for i, o in enumerate(dic):
                                for j, p in enumerate(o):
                                    model.cbLazy(c_var[s,h,i] >= dic[o] * (quicksum(x_var[s, h, k] for k in o[j:]) - quicksum(x_var[s, h_, k] for k in o[j:])))
                for i in range(len(dic)):
                    model.cbLazy(d[i] >= quicksum(c_var[s,h,i] for s in self.S for h in self.H if h != 1) - alpha_var)
                model.cbLazy(u_var >= alpha_var + quicksum(d[i] for i in range(len(dic))) / ((1 - self.beta) * len(self.O)))
        self.model = Model("CuttingPlaneCVaRMinimization")
        a = []
        dic = {}
        for i in self.OC:
            # print(i)
            if not i in a:
                a.append(i)
        for j in a:
            dic[tuple(j)] = self.OC.count(j)
        x, c, d = {}, {}, {}
        for s in self.S:
            for h in self.H:
                for p in self.P:
                    x[s, h, p] = self.model.addVar(vtype = "B", name = "x_" + str(s) + "_" + str(h) + "_" + str(p))

        for s in self.S:
            for h in range(2, len(self.H) + 1):
                for i, o in enumerate(self.O):
                    c[s, h, i] = self.model.addVar(vtype = "C", lb = 0, name = "c_" + str(s) + "_" + str(h) + "_" + str(i))

        for i in range(len(self.O)):
            d[i] = self.model.addVar(vtype = "C", lb = 0)
        alpha = self.model.addVar(vtype = "C", lb = 0, name = "alpha")
        u = self.model.addVar(vtype = "C", lb = 0, name = "u")

        self.model.update()
        self.model._vars = self.model.getVars()

        for p in self.P:
            self.model.addConstr(quicksum(x[s, h, p] for s in self.S for h in self.H) == self.n_init[p - 1])

        for s in self.S:
            for h in self.H:
                self.model.addConstr(quicksum(x[s, h, p] for p in self.P) <= 1)

        for s in self.S:
            for h in range(1, len(self.H)):
                self.model.addConstr(quicksum(x[s, h+1, p] for p in self.P) <= quicksum(x[s, h, p] for p in self.P))

        for s in self.S:
            for h in range(2, len(self.H) + 1):
                for h_ in range(1, h):
                    for i, o in enumerate(dic):
                        for j, p in enumerate(o):
                            self.model.addConstr(c[s, h, i] >= dic[o] * (quicksum(x[s, h, k] for k in o[j:]) - quicksum(x[s, h_, k] for k in o[j:])))
        for i in range(len(dic)):
            self.model.addConstr(d[i] >= quicksum(c[s, h, i] for s in self.S for h in self.H if h != 1) - alpha)
        self.model.addConstr(u >= alpha+quicksum(d[i] for i in range(len(dic))) / ((1-self.beta) * len(self.O)))
        # if len(self.OC)==0:
        #     self.OC=[self.O1]
        # else:
        #     if self.O1 in self.OC:
        #         self.OC=self.OC.remove(self.O1)
        # if len(self.OC)>self.size:
        #     self.OC=self.OC[self.size:]
        # print("OC=",self.OC)
        for s in self.S:
            for h in range(2, len(self.H) + 1):
                for i, o in enumerate([list(self.O1)]):
                    for j, p in enumerate(o):
                        self.model.addConstr(quicksum(x[s, h, k] for k in o[j:]) <= quicksum(x[s, h-1, k] for k in o[j:]))

        for i in range(len([self.O1])):
            self.model.addConstr(d[i] >= quicksum(c[s, h, i] for s in self.S for h in self.H if h != 1) - alpha)

        self.model.addConstr(u >= alpha + quicksum(d[i] for i in range(len(self.O))) / ((1 - self.beta) * len(self.O)))
        self.model.setObjective(u)
        self.model._vars = self.model.getVars()
        self.model.params.LazyConstraints = 1
        # self.model.params.TimeLimit = 3600
        self.model.params.MIPFocus = 2
        if self.f >= self.height:
            self.model.Params.TimeLimit = 7200
            self.model.optimize(add_cutting_plane)
            data_1 = np.random.multivariate_normal(self.mean, self.cov, size = self.size_)

            O_ = np.argsort(data_1)
            OR = (O_ + np.ones((self.size_, self.ship_num)).astype(int)).tolist()
            global penalty_cvar
            penalty_cvar = []
            for k in OR:
                a = 0
                for j in range(self.stack_num):
                    for i in range(1, self.height):
                        for i_ in range(i+1, self.height + 1):
                            if result[i-1][j] != 0 and result[i_-1][j] != 0:
                                if k.index(result[i - 1][j]) > k.index(result[i_ - 1][j]):
                                    a += 1
                                    # if Gamma ==2:
                                        # print(j+1,height-i+1,O)
                                    # print("penalty!")
                                    # print(i,i_,j+1,O)
                                    break
                penalty_cvar.append(a)
            penalty_cvar = np.sort(penalty_cvar)
            penalty_cvar = penalty_cvar[round(self.beta * self.size_):]

        else:
            print("実行不可")

    def get_optimal_val(self):
        if self.model.Status == GRB.OPTIMAL:
            self.LB = self.model.ObjVal
            return self.model.ObjVal
        else:
            return None

    def get_optimal_sol(self):
        if self.model.Status == GRB.OPTIMAL:
            x_opt = {}
            c_opt = {}
            for var in self.model._vars:
                if "x_" in var.VarName:
                    # print(var)
                    s = int(var.VarName.split("_")[-3])
                    h = int(var.VarName.split("_")[-2])
                    p = int(var.VarName.split("_")[-1])
                    x_opt[s, h, p] = var.X
                if "c_" in var.VarName:
                    s = int(var.VarName.split("_")[-3])
                    h = int(var.VarName.split("_")[-2])
                    i = int(var.VarName.split("_")[-1])
                    c_opt[s, h, i] = var.X
                if "alpha" in var.VarName:
                    alpha_opt = var.X
                if "u" in var.VarName:
                    u_opt = var.X
            return (x_opt, c_opt, alpha_opt, u_opt)
        else:
            return None

    def get_result(self, x_tmp):
        self.x_tmp = x_tmp
        EPS = 1.e-6
        self.result = np.zeros((self.height, self.stack_num))
        for (s, h, p) in self.x_tmp:
            if self.x_tmp[s, h, p] > EPS:
                self.result[self.height - h][s - 1] = int(p)
        self.result = self.result.astype(int)
        return self.result

def robust(init:list, Gamma, mean, cov, size_ = 10000):
    # nと初期配置を変更しなければいけない
    num = sum(init[2])

    import numpy as np

    O1 = set(i+1 for i in range(len(init[2])))

    Q = set(i+1 for i in range(init[0]))
    L = set(i+1 for i in range(init[1]))
    P = set(i+1 for i in range(len(init[2])))
    I = set(i+1 for i in range(num))
    f = init[0] * init[1] - len(I)
    a = 1
    gamma = []
    for i in init[2]:
        for j in range(1, i+1):
            gamma.append(a)
        a += 1
    
    m = Model("BI")

    # 変数の定義
    alpha, beta = {}, {}
    for i in I:
        for q in Q:
            alpha[i, q] = m.addVar(vtype = "B")
            beta[i, q] = m.addVar(vtype = "B")
    J = []
    for i in I:
        J.append([])
        for j in I:
            if gamma[i - 1] < gamma[j - 1]:
                if gamma[j - 1] - gamma[i - 1] <= Gamma:
                    J[i - 1].append(j)

    for q in Q:
        m.addConstr(quicksum((alpha[i, q] + beta[i, q]) for i in I) <= len(L))

    for i in I:
        m.addConstr(quicksum((alpha[i, q] + beta[i, q]) for q in Q) == 1)

    for i in I:
        for j in J[i - 1]:
            for q in Q:
                m.addConstr(alpha[i, q] + alpha[j, q] + beta[j, q] <= 1)

    m.setObjective(quicksum(beta[i, q] for i in I for q in Q))

    if f >= init[1]:
        m.Params.TimeLimit = 600
        m.optimize()
    EPS = 1.e-6

    if m.Status == GRB.OPTIMAL:
        print("====================================================")

        EPS = 1.e-6
        a = []
        for q in Q:
            a.append([])
        for (i, q) in alpha:
            if alpha[i, q].X > EPS:
                a[q - 1].append(gamma[i - 1])
        
        for (i, q) in beta:
            if beta[i, q].X > EPS:
                a[q - 1].append(gamma[i - 1])

        for q in Q:
            a[q - 1] = sorted(a[q - 1], reverse = True)

        global result_r
        result_r = np.zeros((init[1],init[0]))
        for q in Q:
            for i, r in enumerate(a[q - 1]):
                result_r[init[1] - i - 1][q - 1] = r
            # print(i,r)

        result_r = result_r.astype(int)
        
        print(result_r)
        print("the objective function", m.objVal)

        np.random.seed()
        data_1 = np.random.multivariate_normal(mean, cov, size = size_)

        O_ = np.argsort(data_1)
        OR = (O_ + np.ones((size_, len(init[2]))).astype(int)).tolist()


        global penalty_r
        penalty_r = []

        for k in OR:
            a = 0
            for j in range(init[0]):
                for i in range(1, init[1]):
                    for i_ in range(i+1, init[1] + 1):
                        if result_r[i-1][j] != 0 and result_r[i_-1][j] != 0:
                            if k.index(result_r[i - 1][j]) > k.index(result_r[i_ - 1][j]):
                                a += 1
                                # if Gamma ==2:
                                    # print(j+1,height-i+1,O)
                                # print("penalty!")
                                # print(i,i_,j+1,O)
                                break
            penalty_r.append(a)
        penalty_r = np.sort(penalty_r)
        penalty_r = penalty_r[round(0.75*size_):]

        # print(penalty_r)

        # import matplotlib.pyplot as plt
        # plt.boxplot(penalty_r)

In [26]:
import numpy as np
from scipy.stats import wishart
from scipy.stats import invwishart

def sample_covariance_matrix_from_wishart(scale_matrix, df):
    # ウィシャート分布から共分散行列をサンプリング
    covariance_matrix = wishart.rvs(df=df, scale=scale_matrix)
    covariance_matrix = np.round(covariance_matrix, 2)
    return covariance_matrix

def generate_random_mean_vector(dim):
    # 平均ベクトルをランダムに生成
    mean_vector = np.random.uniform(0, dim, dim)
    # 昇順に並べ替え
    mean_vector.sort()
    # 小数点以下2桁に丸める
    mean_vector = np.round(mean_vector, 2)
    return mean_vector

def is_positive_definite(matrix):
    # 固有値分解を行う
    eigvals, _ = np.linalg.eig(matrix)
    # すべての固有値が正であるかを確認する
    if np.all(eigvals > 0):
        return True
    else:
        return False

def sample_covariance_matrix(n, df, scale_matrix):
    """
    n: int, サンプルサイズ
    df: int, 自由度
    scale_matrix: array_like, スケール行列
    """
    # 逆ウィシャート分布から共分散行列をサンプリング
    cov_matrix = invwishart.rvs(df, scale_matrix, size=n)
    # 共分散行列の要素を小数点以下2桁に丸める
    rounded_cov_matrix = np.round(cov_matrix, decimals=2)
    return rounded_cov_matrix

In [20]:
size = 10000
size_ = 10000
beta = 0.75
n = 1
S = 4
H = S
inst = [S,H,list(range(1, S)) * 2]
P = len(inst[2])
df = P
scale_matrix = np.eye(P)
mean = generate_random_mean_vector(P)
cov = sample_covariance_matrix(n, df, scale_matrix)

print(mean)
print(cov)

[ 0.97  2.58  3.46  5.99  6.72  6.91  8.36  8.44  8.68  8.73  9.34 11.5
 12.86 13.29]
[[ 0.5  -0.01  0.19  0.31 -0.08  0.08 -0.04  0.01 -0.35 -0.34 -0.28 -0.05
  -0.46 -0.2 ]
 [-0.01  0.41  0.18  0.04 -0.04  0.24 -0.31  0.11 -0.1  -0.02  0.29  0.12
  -0.05 -0.16]
 [ 0.19  0.18  2.72  1.15 -1.36  0.63  0.6   0.28 -1.19 -1.71  1.13  0.15
  -2.62 -1.58]
 [ 0.31  0.04  1.15  0.72 -0.56  0.26  0.31  0.09 -0.72 -0.89  0.21  0.
  -1.32 -0.74]
 [-0.08 -0.04 -1.36 -0.56  0.86 -0.36 -0.38 -0.19  0.6   0.89 -0.59 -0.05
   1.32  0.81]
 [ 0.08  0.24  0.63  0.26 -0.36  0.38 -0.11  0.14 -0.38 -0.35  0.35  0.1
  -0.57 -0.46]
 [-0.04 -0.31  0.6   0.31 -0.38 -0.11  0.84  0.08 -0.14 -0.55  0.04 -0.13
  -0.63 -0.21]
 [ 0.01  0.11  0.28  0.09 -0.19  0.14  0.08  0.22 -0.04 -0.19  0.25  0.07
  -0.15 -0.17]
 [-0.35 -0.1  -1.19 -0.72  0.6  -0.38 -0.14 -0.04  0.98  0.9  -0.15 -0.02
   1.47  0.84]
 [-0.34 -0.02 -1.71 -0.89  0.89 -0.35 -0.55 -0.19  0.9   1.32 -0.46 -0.02
   1.86  1.03]
 [-0.28  0.29  1.13  0.21 -

In [25]:
# model=CVaR(inst, mean, cov, size, size_, beta)
# model.solve()
robust(inst, 1, mean, cov, size_)

Set parameter TimeLimit to value 600
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1912 rows, 896 columns and 7336 nonzeros
Model fingerprint: 0xea5cf89e
Variable types: 0 continuous, 896 integer (896 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Found heuristic solution: objective 29.0000000
Presolve removed 0 rows and 56 columns
Presolve time: 0.03s
Presolved: 1912 rows, 840 columns, 6888 nonzeros
Found heuristic solution: objective 0.0000000
Variable types: 0 continuous, 840 integer (840 binary)

Explored 0 nodes (0 simplex iterations) in 0.05 seconds (0.02 work units)
Thread count was 8 (of 8 available processors)

Solution count 2: 0 29 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.000

In [147]:
print(np.mean(penalty_cvar), np.max(penalty_cvar))

2.034 5


In [148]:
a, b = [], []
for g in range(1, 6):
    robust(inst, g, mean, cov, size_)
    a.append(np.mean(penalty_r))
    b.append(np.max(penalty_r))
print(min(a), np.argmin(a) + 1)
print(min(b), np.argmin(b) + 1)

Set parameter TimeLimit to value 600
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 92 rows, 96 columns and 420 nonzeros
Model fingerprint: 0x79069322
Variable types: 0 continuous, 96 integer (96 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 3.0000000
Presolve removed 0 rows and 12 columns
Presolve time: 0.00s
Presolved: 92 rows, 84 columns, 372 nonzeros
Found heuristic solution: objective 0.0000000
Variable types: 0 continuous, 84 integer (84 binary)

Explored 0 nodes (0 simplex iterations) in 0.04 seconds (0.00 work units)
Thread count was 8 (of 8 available processors)

Solution count 2: 0 3 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%


[[0 3 0 0]
 [2 3 2 0]
 [5 6 4 1]
 [5 6 6 3]]
the objective function 0.0
Set parameter TimeLimit to value 600
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 148 rows, 96 columns and 588 nonzeros
Model fingerprint: 0xb330415d
Variable types: 0 continuous, 96 integer (96 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 3.0000000
Presolve removed 40 rows and 12 columns
Presolve time: 0.01s
Presolved: 108 rows, 84 columns, 504 nonzeros
Found heuristic solution: objective 0.0000000
Variable types: 0 continuous, 84 integer (84 binary)

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.00 work units)
Thread count was 8 (of 8 available processors)

Solution count 2: 0 3 

Optimal solution found (tolerance 1.00e-04)
Best objective

In [149]:
a

[2.1244, 1.3256, 1.3472, 1.4248, 1.046]

In [150]:
b

[4, 2, 2, 2, 2]

## データセット4分布A不良配置数

In [5]:
size = 100
size_ = 10000
beta = 0.75
inst = [10,10,[4,6,3,2,5,8,7,2,5,7,5,6,11,7,5,7]]
pcvar=[]
import numpy as np
mean = np.arange(1, len(inst[2]) + 1, 1)
covl = [1 for i in range(len(inst[2]))]
mean = mean[:len(inst[2])]
covl = covl[:len(inst[2])]
# a=list(a)
# a=random.sample(a,ship_num)
# mean=np.sort(a)
cov = np.zeros((len(inst[2]), len(inst[2])))
for i in range(len(inst[2])):
    for j in range(len(inst[2])):
        if i == j:
            cov[i][i] = covl[i]
for i in range(len(inst[2])):
    for j in range(len(inst[2])):
        if i != j:
            cov[i][j]=0

# model=DirectSolve(inst, mean, cov, size, size_, beta)
# model=CVaR(inst, mean, cov, size, size_, beta)
# model.solve()
            
a = []
for g in range(1, len(inst[2])):
    robust(inst, g, mean, cov, size_)
    a.append(np.mean(penalty_r))
print(min(a), np.argmin(a) + 1)

Set parameter TimeLimit to value 600
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 5010 rows, 1800 columns and 18330 nonzeros
Model fingerprint: 0x86e0aa72
Variable types: 0 continuous, 1800 integer (1800 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 43.0000000
Presolve removed 0 rows and 70 columns
Presolve time: 0.53s
Presolved: 5010 rows, 1730 columns, 17840 nonzeros
Found heuristic solution: objective 0.0000000
Variable types: 0 continuous, 1730 integer (1730 binary)

Explored 0 nodes (0 simplex iterations) in 0.80 seconds (0.05 work units)
Thread count was 8 (of 8 available processors)

Solution count 2: 0 43 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, g

In [16]:
def generate_sequence(n):
    sequence = []
    for i in range(n):
        if i % 3 == 0:
            sequence.append(i // 3 + 1)
        elif i % 3 == 1:
            sequence.append((i // 3) + 1 + 0.5)
        else:
            sequence.append((i // 3) + 2)
    return sequence

n = 10  # 生成する数列の要素数
result_sequence = generate_sequence(n)
print(result_sequence)

[1, 1.5, 2, 2, 2.5, 3, 3, 3.5, 4, 4]


## データセット1

In [17]:
size = 10000
size_ = 10000
beta = 0.75
inst = [4,4,[2,2,2,2,2,2]]
pcvar=[]
import numpy as np
mean = np.array([1, 1.5, 3.5, 4, 6, 6.5, 8.5, 9, 15, 15.5, 18.5, 19, 22, 22.5, 25.5, 26])
mean = mean[:len(inst[2])]
covl = [1, 1.5, 1, 2, 1, 1.5, 1, 2, 1, 1.5,1, 2, 1, 1.5, 1, 2]
covl = covl[:len(inst[2])]
# a=list(a)
# a=random.sample(a,ship_num)
# mean=np.sort(a)
cov = np.zeros((len(inst[2]), len(inst[2])))
for i in range(len(inst[2])):
    for j in range(len(inst[2])):
        if i == j:
            cov[i][i] = covl[i]
for i in range(len(inst[2])):
    for j in range(len(inst[2])):
        if i != j:
            cov[i][j]=0

corr_list = [[1,5,-0.8],[1,4,0.8]]

for i in corr_list:
    cov[i[0]-1][i[1]-1] = i[2]*np.sqrt(cov[i[0]-1][i[0]-1]*cov[i[1]-1][i[1]-1])
    cov[i[1]-1][i[0]-1] = i[2]*np.sqrt(cov[i[0]-1][i[0]-1]*cov[i[1]-1][i[1]-1])

# cov[13][15] = 1.3
# cov[15][13] = 1.3

model=CVaR(inst, mean, cov, size, size_, beta)
model.solve()

# a = []
# for g in range(1, len(inst[2])):
#     robust(inst, g, mean, cov, size_)
#     a.append(np.mean(penalty_r))
# print(min(a), np.argmin(a) + 1)

# robust(inst, 6, mean, cov, size_)

C:\Users\daiki\AppData\Local\Temp\ipykernel_18464\2830071780.py:220: RuntimeWarning: covariance is not positive-semidefinite.
  data_1 = np.random.multivariate_normal(mean, cov, size = self.size)


Set parameter LazyConstraints to value 1
Set parameter MIPFocus to value 2
Set parameter TimeLimit to value 7200
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 109 rows, 130098 columns and 10858 nonzeros
Model fingerprint: 0x89905460
Variable types: 130002 continuous, 96 integer (96 binary)
Coefficient statistics:
  Matrix range     [4e-04, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
[[0 0 0 2]
 [1 3 0 2]
 [3 5 1 4]
 [6 6 4 5]]
5745 1
Presolve removed 21 rows and 8 columns
Presolve time: 0.33s
Presolved: 88 rows, 130090 columns, 10620 nonzeros
Variable types: 130002 continuous, 88 integer (88 binary)
Root relaxation presolved: 88 rows, 130090 columns, 10620 nonzeros


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.700000e+01   0.000000e+00     12s
   

C:\Users\daiki\AppData\Local\Temp\ipykernel_18464\2830071780.py:370: RuntimeWarning: covariance is not positive-semidefinite.
  data_1 = np.random.multivariate_normal(self.mean, self.cov, size = self.size_)


In [20]:
# np.mean(penalty_cvar)
result

array([[0, 1, 0, 2],
       [0, 1, 0, 2],
       [3, 4, 5, 6],
       [3, 4, 5, 6]])

In [19]:
a = []
for g in range(1, len(inst[2])):
    robust(inst, g, mean, cov, size_)
    a.append(np.mean(penalty_r))
print(min(a), np.argmin(a) + 1)

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 96 rows, 96 columns and 432 nonzeros
Model fingerprint: 0x7653d5fb
Variable types: 0 continuous, 96 integer (96 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 4.0000000
Presolve removed 0 rows and 8 columns
Presolve time: 0.01s
Presolved: 96 rows, 88 columns, 400 nonzeros
Found heuristic solution: objective 0.0000000
Variable types: 0 continuous, 88 integer (88 binary)

Explored 0 nodes (0 simplex iterations) in 0.13 seconds (0.00 work units)
Thread count was 8 (of 8 available processors)

Solution count 2: 0 4 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%
[[1 0 0 1]
 [3 0 0 4]
 [3 2 2 4]
 [5 5 6 6]]
the 

C:\Users\daiki\AppData\Local\Temp\ipykernel_18464\2830071780.py:522: RuntimeWarning: covariance is not positive-semidefinite.
  data_1 = np.random.multivariate_normal(mean, cov, size = size_)


Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 160 rows, 96 columns and 624 nonzeros
Model fingerprint: 0x14f73017
Variable types: 0 continuous, 96 integer (96 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 4.0000000
Presolve removed 48 rows and 8 columns
Presolve time: 0.01s
Presolved: 112 rows, 88 columns, 528 nonzeros
Found heuristic solution: objective 3.0000000
Variable types: 0 continuous, 88 integer (88 binary)

Root relaxation: objective 0.000000e+00, 24 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       0.0000000    0.00000  0.00%     -    0s


## データセット2

In [24]:
size = 10000
size_ = 10000
beta = 0.75
inst = [6,6,[2 for i in range(15)]]
pcvar=[]
import numpy as np
mean = np.array([1, 1.5, 3.5, 4, 6, 6.5, 8.5, 9, 11, 11.5, 13.5, 14, 16, 16.5, 18.5, 19])
mean = mean[:len(inst[2])]
covl = [1, 1.5, 1, 2, 1, 1.5, 1, 2, 1, 1.5,1, 2, 1, 1.5, 1, 2]
covl = covl[:len(inst[2])]
# a=list(a)
# a=random.sample(a,ship_num)
# mean=np.sort(a)
cov = np.zeros((len(inst[2]), len(inst[2])))
for i in range(len(inst[2])):
    for j in range(len(inst[2])):
        if i == j:
            cov[i][i] = covl[i]
for i in range(len(inst[2])):
    for j in range(len(inst[2])):
        if i != j:
            cov[i][j]=0

corr_list = [[1,5,-0.8],[1,4,0.8]]

for i in corr_list:
    cov[i[0]-1][i[1]-1] = i[2]*np.sqrt(cov[i[0]-1][i[0]-1]*cov[i[1]-1][i[1]-1])
    cov[i[1]-1][i[0]-1] = i[2]*np.sqrt(cov[i[0]-1][i[0]-1]*cov[i[1]-1][i[1]-1])

# cov[13][15] = 1.3
# cov[15][13] = 1.3

model=CVaR(inst, mean, cov, size, size_, beta)
model.solve()

# a = []
# for g in range(1, len(inst[2])):
#     robust(inst, g, mean, cov, size_)
#     a.append(np.mean(penalty_r))
# print(min(a), np.argmin(a) + 1)

# robust(inst, 6, mean, cov, size_)

C:\Users\daiki\AppData\Local\Temp\ipykernel_18464\2830071780.py:220: RuntimeWarning: covariance is not positive-semidefinite.
  data_1 = np.random.multivariate_normal(mean, cov, size = self.size)


Set parameter LazyConstraints to value 1
Set parameter MIPFocus to value 2
Set parameter TimeLimit to value 7200
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 534 rows, 310542 columns and 19216 nonzeros
Model fingerprint: 0x8da0cc1a
Variable types: 310002 continuous, 540 integer (540 binary)
Coefficient statistics:
  Matrix range     [4e-04, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Presolve removed 67 rows and 36 columns
Presolve time: 1.77s
Presolved: 467 rows, 310506 columns, 17348 nonzeros
Variable types: 310002 continuous, 504 integer (504 binary)
Root relaxation presolved: 467 rows, 310506 columns, 17348 nonzeros


Use crossover to convert LP symmetric solution to basic solution...

Root relaxation: objective 0.000000e+00, 359 iterations, 1.34 seconds (0.09 work units)
Total elapsed time = 6.55s
[[ 2  1  0  0

C:\Users\daiki\AppData\Local\Temp\ipykernel_18464\2830071780.py:370: RuntimeWarning: covariance is not positive-semidefinite.
  data_1 = np.random.multivariate_normal(self.mean, self.cov, size = self.size_)


In [25]:
np.mean(penalty_cvar)

0.0016

In [26]:
a = []
for g in range(1, len(inst[2])):
    robust(inst, g, mean, cov, size_)
    a.append(np.mean(penalty_r))
print(min(a), np.argmin(a) + 1)

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 372 rows, 360 columns and 1728 nonzeros
Model fingerprint: 0x86e34cce
Variable types: 0 continuous, 360 integer (360 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+00]
Found heuristic solution: objective 17.0000000
Presolve removed 0 rows and 12 columns
Presolve time: 0.03s


Presolved: 372 rows, 348 columns, 1680 nonzeros
Found heuristic solution: objective 0.0000000
Variable types: 0 continuous, 348 integer (348 binary)

Explored 0 nodes (0 simplex iterations) in 0.07 seconds (0.00 work units)
Thread count was 8 (of 8 available processors)

Solution count 2: 0 17 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%
[[ 2  1  1  0  0  0]
 [ 2  5  5 10  0  0]
 [ 4  8  7 10  0  3]
 [ 6  8  7 12  4  3]
 [ 6 11  9 15  9 11]
 [12 14 13 15 14 13]]
the objective function 0.0


C:\Users\daiki\AppData\Local\Temp\ipykernel_18464\2830071780.py:522: RuntimeWarning: covariance is not positive-semidefinite.
  data_1 = np.random.multivariate_normal(mean, cov, size = size_)


Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 684 rows, 360 columns and 2664 nonzeros
Model fingerprint: 0x1949b436
Variable types: 0 continuous, 360 integer (360 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+00]
Found heuristic solution: objective 19.0000000
Presolve removed 228 rows and 12 columns
Presolve time: 0.03s
Presolved: 456 rows, 348 columns, 2316 nonzeros
Found heuristic solution: objective 2.0000000
Variable types: 0 continuous, 348 integer (348 binary)

Root relaxation: objective 0.000000e+00, 102 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

H    0     0                       0.0000000    0.00000  0.00% 

In [26]:
result_r

array([[ 0,  0,  1,  5,  0,  0,  0,  6],
       [ 0,  3,  1,  5,  0,  4,  2,  6],
       [ 3,  3,  1,  5,  0,  4,  2,  6],
       [ 3,  9,  1,  5,  1,  4,  2,  6],
       [ 3,  9,  7, 11,  1,  4,  8, 12],
       [10,  9,  7, 11, 10,  4,  8, 12],
       [10,  9, 13, 11, 10,  4,  8, 12],
       [10,  9, 13, 11, 10, 13,  8, 12]])